# Data Processing

In [ ]:
import os
import pandas as pd
import ast  # For safely evaluating array-like strings
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('wordnet')

def clean_data(df, folder_name):
    """
    Perform data cleaning based on folder-specific requirements.

    Parameters:
    df (pd.DataFrame): The DataFrame to clean.
    folder_name (str): The name of the folder to apply specific cleaning rules.

    Returns:
    pd.DataFrame: The cleaned DataFrame.
    """
    # General cleaning logic
    df = df.copy()
    
    # Apply folder-specific cleaning logic
    if folder_name == "080_Books":
        columns_fill_zero = ['Copies Left', 'Wished Users', 'Reviews']
        df[columns_fill_zero] = df[columns_fill_zero].fillna(0)
        numerical_columns = ['Ratings']
        df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())
        categorical_columns = ['Publication']
        df[categorical_columns] = df[categorical_columns].fillna('Unknown')
        text_columns = ['Book Title', 'Author', 'Category', 'Stock Status', 'Edition', 'Publication']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df['Price (TK)'] = pd.to_numeric(df['Price (TK)'], errors='coerce')

    elif folder_name == "079_Coffee":
        text_columns = ['store_location', 'product_category', 'product_type', 'product_detail', 'Month_1', 'Weekday_1']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df['Revenue'] = df['Revenue'].astype(str).str.replace(',', '.').str.strip()
        numeric_columns = ['transaction_qty', 'unit_price']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    elif folder_name == "078_Fires":
        text_columns = ['calendar_names_1', 'calendar_names_2', 'calendar_1', 'calendar_2']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        numeric_columns = ['area', 'DMC', 'DC', 'temp', 'ISI', 'wind']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    elif folder_name == "076_NBA":
        text_columns = ['Season_type', 'PLAYER', 'TEAM']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    elif folder_name == "075_Mortality":
        text_columns = ['Region', 'Status', 'Sex', 'Cause']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    elif folder_name == "074_Lift":
        text_columns = ['Lifter Name', 'Weight Class', 'Lift Type']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    # Add other folder-specific cleaning logic here
    # Folder-specific cleaning logic
# Folder-specific cleaning logic
    elif folder_name == "071_COL":
        text_columns = ['Country']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    # Apply folder-specific cleaning logic
    elif folder_name == "070_OpenFoodFacts":
        # Handle Missing Values
        for col in df.columns:
            if df[col].dtype == "object":
                df[col] = df[col].fillna("unknown")
            else:
                df[col] = df[col].fillna(0)

        # Standardize Text Data
        object_columns = df.select_dtypes(include=["object"]).columns
        for col in object_columns:
            df[col] = df[col].str.lower().str.strip()

        # Handle Array-Like Data
        for col in df.columns:
            if df[col].dtype == "object":
                def process_array(value):
                    try:
                        array = ast.literal_eval(value)
                        if isinstance(array, list):
                            if not array:
                                return ["unknown"]
                            return [str(item).lower().strip() if isinstance(item, str) else item for item in array]
                        else:
                            return value
                    except (ValueError, SyntaxError):
                        return value
                df[col] = df[col].apply(process_array)

        # Ensure Correct Data Types
        for col in df.columns:
            if df[col].dtype == "object":
                try:
                    df[col] = pd.to_numeric(df[col], errors="ignore")
                except ValueError:
                    pass

        # Process Specific Columns
        def process_column(entry):
            if isinstance(entry, list):
                return [item.strip().lower() for item in entry if isinstance(item, str)]
            elif isinstance(entry, str):
                entry = entry.strip('[]')  # Remove the outer brackets
                return [item.strip().lower() for item in entry.split(',') if item.strip()]
            else:
                return entry

        columns_to_process = [
            'categories_en', 'states_en', 'brands', 'labels_en', 'stores',
            'countries_en', 'ingredients_analysis_tags', 'ingredients_tags'
        ]

        for column in columns_to_process:
            if column in df.columns:
                df[column] = df[column].apply(process_column)

    elif folder_name == "069_Taxonomy":
        df.drop(columns=['Unnamed: 7'], inplace=True, errors='ignore')
        df.dropna(subset=['Unique ID', 'Parent'], inplace=True)
        for column in ['Tier 2', 'Tier 3', 'Tier 4']:
            if column in df.columns and pd.api.types.is_categorical_dtype(df[column]):
                df[column] = df[column].cat.add_categories('Unknown')
                df[column] = df[column].fillna('Unknown')

        text_columns = ['Name', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df.drop_duplicates(inplace=True)

    elif folder_name == "068_WorldBank_Awards":
        # Ensure the 'Procurement Method' column is handled as a categorical column
        if 'Procurement Method' in df.columns:
            if pd.api.types.is_categorical_dtype(df['Procurement Method']):
                 # Add 'Unknown' category if not already present
                 if 'Unknown' not in df['Procurement Method'].cat.categories:
                      df['Procurement Method'] = df['Procurement Method'].cat.add_categories('Unknown')
            df['Procurement Method'] = df['Procurement Method'].fillna('Unknown')

        # Ensure 'Project Global Practice' column is handled as a categorical column
        if 'Project Global Practice' in df.columns:
            if pd.api.types.is_categorical_dtype(df['Project Global Practice']):
                # Add 'Not Specified' category if not already present
                if 'Not Specified' not in df['Project Global Practice'].cat.categories:
                     df['Project Global Practice'] = df['Project Global Practice'].cat.add_categories('Not Specified')
            df['Project Global Practice'] = df['Project Global Practice'].fillna('Not Specified')

        
        if 'Borrower Contract Reference Number' in df.columns:
            df['Borrower Contract Reference Number'] = df['Borrower Contract Reference Number'].cat.add_categories('N/A')
            df['Borrower Contract Reference Number'] = df['Borrower Contract Reference Number'].fillna('N/A')

        df['Supplier ID'] = df['Supplier ID'].fillna(-1)
        df['Contract Description'] = df['Contract Description'].fillna('No Description Provided')
        if 'Supplier Country Code' in df.columns:
            df['Supplier Country Code'] = df['Supplier Country Code'].cat.add_categories('Unknown')
            df['Supplier Country Code'] = df['Supplier Country Code'].fillna('Unknown')

        if 'Supplier Country' in df.columns and pd.api.types.is_categorical_dtype(df['Supplier Country']):
             df['Supplier Country'] = df['Supplier Country'].cat.add_categories('Unknown')
             df['Supplier Country'] = df['Supplier Country'].fillna('Unknown')

        if 'Supplier' in df.columns and pd.api.types.is_categorical_dtype(df['Supplier']):
             df['Supplier'] = df['Supplier'].cat.add_categories('Unknown Supplier')
             df['Supplier'] = df['Supplier'].fillna('Unknown Supplier')

        df.drop(columns=['Borrower Country Code'], inplace=True, errors='ignore')
        text_columns = [
            'Procurement Method', 'Project Global Practice', 'Contract Description',
            'Supplier Country Code', 'Borrower Country', 'Region',
            'Supplier Country', 'Supplier', 'Project Name'
        ]
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df['Region'] = df['Region'].str.replace('_', ' ')
        df.drop_duplicates(inplace=True)

    elif folder_name == "067_TripAdvisor":
        def process_text(text):
            lemmatizer = WordNetLemmatizer()
            #stemmer = PorterStemmer() ####### Bad dite paren
            text = text.lower().strip()
            tokens = word_tokenize(text)
            return " ".join([
                lemmatizer.lemmatize(token)  # Only lemmatize the tokens
                #stemmer.stem(lemmatizer.lemmatize(token))
                for token in tokens if token not in ENGLISH_STOP_WORDS
            ])
        
        # Ensure 'date_stayed' is processed correctly
        if 'date_stayed' in df.columns:
            if pd.api.types.is_categorical_dtype(df['date_stayed']):
                df['date_stayed'] = df['date_stayed'].cat.add_categories(['Unknown'])
            df['date_stayed'] = df['date_stayed'].fillna('Unknown')
        
        # Process other text columns
        if 'title' in df.columns:
            df['title'] = df['title'].astype(str).fillna("").str.lower().str.strip()

        if 'ratings' in df.columns:
            df['ratings'] = df['ratings'].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)
        if 'text' in df.columns:
            df['text'] = df['text'].apply(process_text)

    elif folder_name == "066_IBM_HR":
        text_columns = df.select_dtypes(include=['object']).columns
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    # Add other folder-specific cleaning logic here

    return df

def process_parquet_files(base_folder):
    """
    Process parquet files for each folder: read, clean, and save as CSV.

    Parameters:
    base_folder (str): Path to the competition folder.

    Returns:
    None
    """
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        
        if os.path.isdir(folder_path):
            print(f"Processing folder: {folder_name}")
            
            # Process 'all.parquet'
            all_parquet_path = os.path.join(folder_path, "all.parquet")
            all_csv_path = os.path.join(folder_path, "cleaned_all.csv")
            if os.path.exists(all_parquet_path):
                all_df = pd.read_parquet(all_parquet_path)
                cleaned_all_df = clean_data(all_df, folder_name)
                cleaned_all_df.to_csv(all_csv_path, index=False)
                print(f"Processed and saved: {all_csv_path}")
            
            # Process 'sample.parquet'
            sample_parquet_path = os.path.join(folder_path, "sample.parquet")
            sample_csv_path = os.path.join(folder_path, "cleaned_sample.csv")
            if os.path.exists(sample_parquet_path):
                sample_df = pd.read_parquet(sample_parquet_path)
                cleaned_sample_df = clean_data(sample_df, folder_name)
                cleaned_sample_df.to_csv(sample_csv_path, index=False)
                print(f"Processed and saved: {sample_csv_path}")

# Usage
base_folder = r"C:\Users\ASUS\Downloads\competition\competition" # Replace with your actual path
process_parquet_files(base_folder)


# Hope little rockstar (score -8.49)

In [ ]:
pip install transformers sentence-transformers pandas numpy torch

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from difflib import get_close_matches
from transformers import pipeline
import warnings
import torch
from concurrent.futures import ProcessPoolExecutor
import zipfile
import logging

# ========== CONFIGURATION ==========
warnings.filterwarnings("ignore", category=FutureWarning)
DEVICE = 0 if torch.cuda.is_available() else -1
MODEL_NAME = "deepset/bert-base-cased-squad2"
DATA_DIR = "/kaggle/input/d/muhammedjunayed/competition-csv/competition_csv/competition"
QA_FILE = os.path.join(DATA_DIR, "test_qa.csv")

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

print("✅ Loading BERT-based Question Answering pipeline...")
qa_pipeline = pipeline("question-answering", model=MODEL_NAME, device=DEVICE)

# ========== TEXT UTILITY FUNCTIONS ==========
def normalize_text(text):
    """Normalize text for matching operations"""
    return re.sub(r'[^\w\s]', '', str(text).lower())

def find_best_column(question, columns, df):
    """Find the best matching column using multiple strategies"""
    question = normalize_text(question)
    columns_lower = [normalize_text(col) for col in columns]

    # Try exact match first
    for col, col_lower in zip(columns, columns_lower):
        if col_lower in question:
            return col if col in df.columns else None

    # Use difflib for approximate matching
    matches = get_close_matches(question, columns_lower, n=1, cutoff=0.7)
    return columns[columns_lower.index(matches[0])] if matches else None

def extract_number(question):
    """Extract numbers from question text. Return 0 if no number is found."""
    numbers = re.findall(r'\d+\.?\d*', question)
    return float(numbers[0]) if numbers else 0

# ========== QUESTION HANDLER ==========
class QuestionHandler:
    def __init__(self, df):
        # Convert columns to numeric where possible
        self.df = df.copy()
        for col in self.df.columns:
            self.df[col] = pd.to_numeric(self.df[col], errors='ignore')
        self.original_columns = self.df.columns.tolist()
        self.numeric_columns = self.df.select_dtypes(include=[np.number]).columns.tolist()

    def handle(self, question):
        """Main entry point for handling questions"""
        question = normalize_text(question)
        handlers = [self.handle_boolean, self.handle_category, self.handle_number, self.handle_list]

        for handler in handlers:
            result = handler(question)
            if result is not None:
                return result

        return "Unhandled question type"

    def handle_boolean(self, question):
        """Handle Yes/No questions"""
        if re.search(r'\b(is|are|does|do|was|were|has|have|had|can|could|should|will|would)\b', question, re.IGNORECASE):
            column = find_best_column(question, self.original_columns, self.df)
            if column and column in self.numeric_columns:
                value = extract_number(question)
                if re.search(r'\b(greater|more than|above)\b', question):
                    return "Yes" if (self.df[column] > value).any() else "No"
                elif re.search(r'\b(less than|below)\b', question):
                    return "Yes" if (self.df[column] < value).any() else "No"
            return "Yes" if any(normalize_text(str(val)) in question for val in self.df.values.flatten()) else "No"
        return None

    def handle_category(self, question):
        """Handle questions expecting a single category value"""
        if re.search(r'\b(which|what|who)\b', question):
            column = find_best_column(question, self.original_columns, self.df)
            if column:
                if column in self.numeric_columns:
                    return str(self.df[column].mode()[0])
                else:
                    return str(self.df[column].mode()[0])
        return None

    def handle_number(self, question):
        """Handle numerical questions"""
        if re.search(r'\b(how many|number of|count|sum|total|average|mean|maximum|minimum|median)\b', question):
            column = find_best_column(question, self.original_columns, self.df)
            if column and column in self.numeric_columns:
                if re.search(r'\b(sum|total)\b', question):
                    return str(int(self.df[column].sum()))
                elif re.search(r'\b(average|mean)\b', question):
                    return f"{self.df[column].mean():.2f}"
                elif re.search(r'\b(maximum|max)\b', question):
                    return str(int(self.df[column].max()))
                elif re.search(r'\b(minimum|min)\b', question):
                    return str(int(self.df[column].min()))
                elif re.search(r'\b(median)\b', question):
                    return f"{self.df[column].median():.2f}"
                else:
                    return str(int(self.df[column].count()))
        return None

    def handle_list(self, question):
        """Handle list-type questions"""
        if re.search(r'\b(list|unique|top|most common)\b', question):
            column = find_best_column(question, self.original_columns, self.df)
            if column:
                if re.search(r'\b(top|most common)\b', question):
                    values = self.df[column].value_counts().index.tolist()[:5]
                else:
                    values = self.df[column].dropna().unique().tolist()[:5]

                if column in self.numeric_columns:
                    values = [float(v) if isinstance(v, (int, float)) else v for v in values]
                    return str(values)
                else:
                    return str(values)
        return None

# ========== MAIN PROCESSING ==========
def process_qa_file(output_filename, file_type, max_rows=None):
    """
    Process QA predictions with enhanced answer formatting
    """
    qa_df = pd.read_csv(QA_FILE)
    
    if max_rows:
        qa_df = qa_df.head(max_rows)

    predictions = []

    for _, row in qa_df.iterrows():
        dataset_name = row['dataset']
        question = row['question']

        # Load the dataset
        dataset_path = os.path.join(DATA_DIR, dataset_name, f"{file_type}.csv")
        if not os.path.exists(dataset_path):
            predictions.append("Dataset not found")
            continue

        df = pd.read_csv(dataset_path, nrows=500)
        handler = QuestionHandler(df)

        # Handle rule-based processing first
        rule_based_answer = handler.handle(question)

        if rule_based_answer and "Unhandled" not in rule_based_answer:
            predictions.append(str(rule_based_answer))
        else:
            # Fallback to QA model with type-aware processing
            try:
                context = df.head(50).to_string(index=False)
                model_answer = qa_pipeline({"question": question, "context": context})
                answer_text = model_answer['answer']
                
                # Boolean answer post-processing
                if re.search(r'^\s*(Is|Does|Do|Are|Was|Were|Has|Have|Had|Can|Could|Should|Will|Would)', 
                            question, re.IGNORECASE):
                    if answer_text.lower() in ['yes', 'no']:
                        final_answer = answer_text.capitalize()
                    else:
                        exists = any(answer_text.lower() in str(cell).lower() 
                                   for cell in df.values.flatten())
                        final_answer = "Yes" if exists else "No"
                else:
                    final_answer = answer_text
                
                predictions.append(final_answer)
            except Exception as e:
                logging.error(f"Error processing question: {question}. Error: {str(e)}")
                predictions.append(f"Error: {str(e)}")

    # Save predictions with proper formatting
    output_file = f"{output_filename}.txt"
    with open(output_file, "w") as f:
        f.write("\n".join(predictions))

    print(f"✅ Predictions saved to {output_file}")

if __name__ == "__main__":
    # Process full dataset
    process_qa_file("predictions", "cleaned_all")
    
    # Process lite version
    process_qa_file("predictions_lite", "cleaned_sample", max_rows=20)
    
    # Create archive
    with zipfile.ZipFile("hope_little_rockstar.zip", "w") as zipf:
        zipf.write("predictions.txt")
        zipf.write("predictions_lite.txt")

    print("✅ Archive.zip created successfully")